# Preserving feature information in DataFrames

This notebook highlights the ability of scikit-mol transformers to return data in DataFrames with meaningful column names. Some use-cases of this feature are illustrated.

***NOTE***: The goal of this notebook is to highlight the advantages of storing transformer output in DataFrames with meaningful column names. This notebook should *not* be considered a set of good practices for training and evaluating QSAR pipelines. The performance metrics of the resulting pipelines are pretty bad: the dataset they have been trained on is pretty small. Tuning the hyperparameters of the Random Forest regressor model (maximum depth of the trees, maximum features to consider when splitting...) can be beneficial. Also including dimensionality reduction / feature selection techniques can be beneficial, since pipelines use an high number of features for a small number of samples. Of course, to further reduce the risk of overfitting, the best hyperparameters and preprocessing techniques should be chosen in cross validation.

In [1]:
from pathlib import Path
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import FunctionTransformer
from scikit_mol.conversions import SmilesToMolTransformer
from sklearn.compose import make_column_selector, make_column_transformer
from scikit_mol.standardizer import Standardizer
from scikit_mol.descriptors import MolecularDescriptorTransformer
from scikit_mol.fingerprints import MorganFingerprintTransformer

In [2]:
csv_file = Path("../tests/data/SLC6A4_active_excapedb_subset.csv")
assert csv_file.is_file()
data = pd.read_csv(csv_file)
data.drop_duplicates(subset="Ambit_InchiKey", inplace=True)

Let's split the dataset in training and test, so we will be able to use the test set to evaluate the performance of models trained on the training set.

In [3]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)

In [4]:
column_smiles = "SMILES"
column_target = "pXC50"

smis_train = data_train.loc[:, column_smiles]
target_train = data_train.loc[:, column_target]
smis_test = data_test.loc[:, column_smiles]
target_test = data_test.loc[:, column_target]

## Descriptors pipeline that returns DataFrames

Define a pipeline that:

- converts SMILES strings to Mol objects
- standardizes the molecules
- computes molecular descriptors

Then, we will configure the pipeline to return output in Pandas DataFrames.
The column names will correspond to the descriptor names.

In [5]:
descriptors_pipeline = make_pipeline(
    SmilesToMolTransformer(),
    Standardizer(),
    MolecularDescriptorTransformer(),
)
descriptors_pipeline.set_output(transform="pandas")

Pipeline(steps=[('smilestomoltransformer', SmilesToMolTransformer()),
                ('standardizer', Standardizer()),
                ('moleculardescriptortransformer',
                 MolecularDescriptorTransformer(desc_list=['MaxAbsEStateIndex',
                                                           'MaxEStateIndex',
                                                           'MinAbsEStateIndex',
                                                           'MinEStateIndex',
                                                           'qed', 'SPS',
                                                           'MolWt',
                                                           'HeavyAtomMolWt',
                                                           'ExactMolWt',
                                                           'NumValenceElectrons',
                                                           'NumRadicalElectrons',
                                                           'MaxPartialCharge',
                                                           'MinPartialCharge',
                                                           'MaxAbsPartialCharge',
                                                           'MinAbsPartialCharge',
                                                           'FpDensityMorgan1',
                                                           'FpDensityMorgan2',
                                                           'FpDensityMorgan3',
                                                           'BCUT2D_MWHI',
                                                           'BCUT2D_MWLOW',
                                                           'BCUT2D_CHGHI',
                                                           'BCUT2D_CHGLO',
                                                           'BCUT2D_LOGPHI',
                                                           'BCUT2D_LOGPLOW',
                                                           'BCUT2D_MRHI',
                                                           'BCUT2D_MRLOW',
                                                           'AvgIpc', 'BalabanJ',
                                                           'BertzCT', 'Chi0', ...]))])

In [6]:
df_descriptors = descriptors_pipeline.transform(smis_train)
df_descriptors

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,13.448610,13.448610,0.056985,-0.432587,0.353101,14.289474,522.592,490.336,522.233014,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12.863074,12.863074,0.026212,-0.050849,0.682187,16.033333,425.558,398.342,425.188546,158.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,13.424788,13.424788,0.266700,-0.413763,0.443905,15.852941,465.588,432.324,465.259169,180.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12.725824,12.725824,0.052996,-0.052996,0.577709,17.812500,478.468,445.204,477.206216,174.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,6.356910,6.356910,0.898244,0.898244,0.658108,13.052632,246.313,232.201,246.115698,92.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,6.217065,6.217065,0.175664,0.175664,0.916154,35.700000,312.240,293.088,311.084370,108.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155,9.458345,9.458345,0.420312,0.420312,0.378112,21.714286,465.645,430.365,465.289246,180.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156,13.267371,13.267371,0.300870,-4.299737,0.919340,23.565217,328.378,305.194,328.176248,128.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157,6.238476,6.238476,0.127623,-0.127623,0.918995,19.428571,323.223,307.095,322.063968,110.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


All scikit-mol transformers are now compatible with the scikit-learn [set_output API](https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_set_output.html).

Let's define a pipeline that returns Morgan fingerprints in a DataFrame.
Columns will be named with the pattern `fp_morgan_1`, `fp_morgan_2`, ...,`fp_morgan_N`.

In [7]:
fingerprints_pipeline = make_pipeline(
    SmilesToMolTransformer(),
    Standardizer(),
    MorganFingerprintTransformer(),
)
fingerprints_pipeline.set_output(transform="pandas")

Pipeline(steps=[('smilestomoltransformer', SmilesToMolTransformer()),
                ('standardizer', Standardizer()),
                ('morganfingerprinttransformer',
                 MorganFingerprintTransformer())])

In [8]:
df_fingerprints = fingerprints_pipeline.transform(smis_train)
df_fingerprints

,fp_morgan_1,fp_morgan_2,fp_morgan_3,fp_morgan_4,fp_morgan_5,fp_morgan_6,fp_morgan_7,fp_morgan_8,fp_morgan_9,fp_morgan_10,...,fp_morgan_2039,fp_morgan_2040,fp_morgan_2041,fp_morgan_2042,fp_morgan_2043,fp_morgan_2044,fp_morgan_2045,fp_morgan_2046,fp_morgan_2047,fp_morgan_2048
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
155,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
157,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Analyze feature importance of regression pipeline

Making the transformation steps return Pandas DataFrames instead of NumPy arrays makes it easy to analyze the feature importance of regression models.

Let's define a pipeline that, starting from SMILES strings, computes descriptors and uses them to predict the target with a Random Forest (RF) regression model. Since descriptors values have very different ranges, it's better to scale them before passing them to the RF regression model.

In [9]:
params_random_forest = {
    "max_depth": 5, # Setting a low maximum depth to avoid overfitting
}

regression_pipeline = make_pipeline(
    SmilesToMolTransformer(),
    Standardizer(),
    MolecularDescriptorTransformer(),
    StandardScaler(), # Scale the descriptors
    RandomForestRegressor(**params_random_forest),
)
regression_pipeline.set_output(transform="pandas")

Pipeline(steps=[('smilestomoltransformer', SmilesToMolTransformer()),
                ('standardizer', Standardizer()),
                ('moleculardescriptortransformer',
                 MolecularDescriptorTransformer(desc_list=['MaxAbsEStateIndex',
                                                           'MaxEStateIndex',
                                                           'MinAbsEStateIndex',
                                                           'MinEStateIndex',
                                                           'qed', 'SPS',
                                                           'MolWt',
                                                           'HeavyAtomMolWt',
                                                           'ExactMolWt',
                                                           'NumValenceElectrons',
                                                           'NumRadicalElectrons',
                                                           'MaxPa...
                                                           'MaxAbsPartialCharge',
                                                           'MinAbsPartialCharge',
                                                           'FpDensityMorgan1',
                                                           'FpDensityMorgan2',
                                                           'FpDensityMorgan3',
                                                           'BCUT2D_MWHI',
                                                           'BCUT2D_MWLOW',
                                                           'BCUT2D_CHGHI',
                                                           'BCUT2D_CHGLO',
                                                           'BCUT2D_LOGPHI',
                                                           'BCUT2D_LOGPLOW',
                                                           'BCUT2D_MRHI',
                                                           'BCUT2D_MRLOW',
                                                           'AvgIpc', 'BalabanJ',
                                                           'BertzCT', 'Chi0', ...])),
                ('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor(max_depth=5))])

In [10]:
regression_pipeline.fit(smis_train, target_train)
pred_test = regression_pipeline.predict(smis_test)

Let's define a simple function to compute regression metrics, and use it to evaluate the test set performance of the pipeline.

In [11]:
def compute_metrics(y_true, y_pred):
    result = {
        "RMSE": mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False),
        "MAE": mean_absolute_error(y_true=y_true, y_pred=y_pred),
        "R2": r2_score(y_true=y_true, y_pred=y_pred),
    }
    return result

performance = compute_metrics(y_true=target_test, y_pred=pred_test)
performance

/home/enrico/anaconda3/envs/scikit-mol/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'RMSE': 0.8635890248250578,
 'MAE': 0.7171077690588582,
 'R2': 0.14233136102782729}

In [12]:
regressor = regression_pipeline[-1]
regressor

RandomForestRegressor(max_depth=5)

Since we used `set_output(transform="pandas")` on the pipeline, the last step of the pipeline (the regression model) has the descriptor names in the `feature_names_in_` attribute. We can use them and the `feature_importances_` attribute to easily analyze the feature importances.

In [13]:
df_importance = pd.DataFrame({"feature": regressor.feature_names_in_, "importance": regressor.feature_importances_})
df_importance

,feature,importance
0,MaxAbsEStateIndex,0.004793
1,MaxEStateIndex,0.003297
2,MinAbsEStateIndex,0.007317
3,MinEStateIndex,0.001835
4,qed,0.003801
...,...,...
205,fr_thiazole,0.000000
206,fr_thiocyan,0.000000
207,fr_thiophene,0.001274
208,fr_unbrch_alkane,0.000212


Sort the features by most to least important:

In [14]:
df_importance.sort_values(by="importance", ascending=False, inplace=True, ignore_index=True)
df_importance

,feature,importance
0,PEOE_VSA6,0.176456
1,MaxAbsPartialCharge,0.054179
2,VSA_EState5,0.033652
3,VSA_EState6,0.032222
4,HallKierAlpha,0.029987
...,...,...
205,fr_hdrzine,0.000000
206,fr_hdrzone,0.000000
207,fr_imidazole,0.000000
208,fr_imide,0.000000


In [15]:
n_top_features = 5
top_features = df_importance.head(n_top_features).loc[:, "feature"].tolist()
print(f"The {n_top_features} most important features are:")
for feature in top_features:
    print(feature)

The 5 most important features are:
PEOE_VSA6
MaxAbsPartialCharge
VSA_EState5
VSA_EState6
HallKierAlpha


## Including external features

The ability to keep the results of scikit-learn transformers in DataFrames with meaningful column names simplifies the task of analyzing the resulting models.

Another good use-case is when we want to combine cheminformatics features from some other tool (QM packages, Deep Learning embeddings...) with the traditional cheminformatics features available in scikit-mol. It will be easier to keep track of the features that come from scikit-mol and the features that come from other tools, if they are stored in DataFrames with meaningful column names.

Let's include features from the popular [CDDD](https://github.com/jrwnter/cddd) tool. CDDD is a Variational AutoEncoder Deep Learning model, and the CDDD features are the inner latent space representations of the SMILES. For additional details, have a look at the original CDDD paper:

> R. Winter, F. Montanari, F. Noé, and D.-A. Clevert, “Learning continuous and data-driven molecular descriptors by translating equivalent chemical representations,” Chem. Sci., vol. 10, no. 6, pp. 1692–1701, Feb. 2019, [doi: 10.1039/C8SC04175J](https://doi.org/10.1039/C8SC04175J).

We have precomputed these features and stored them in a file:

In [16]:
file_cddd_features = Path("../tests/data/CDDD_SLC6A4_active_excapedb_subset.csv.gz")
assert file_cddd_features.is_file()
df_cddd = pd.read_csv(file_cddd_features)
df_cddd

,Unnamed: 0,Ambit_InchiKey,cddd_1,cddd_2,cddd_3,cddd_4,cddd_5,cddd_6,cddd_7,cddd_8,...,cddd_503,cddd_504,cddd_505,cddd_506,cddd_507,cddd_508,cddd_509,cddd_510,cddd_511,cddd_512
0,0,RBCQCVSMIQCOMN-PCQZLOAONA-N,0.034484,0.288955,-0.038534,0.485996,0.398606,-0.077340,0.417946,-0.114399,...,-0.442085,-0.122121,-0.755012,0.580011,-0.999539,-0.466382,0.378511,0.579342,0.753847,0.812943
1,1,ALZTYVXVRZIERJ-UHFFFAOYNA-N,-0.784729,0.146379,0.442466,0.187816,0.042911,-0.007460,0.012899,0.170997,...,-0.434862,0.216556,-0.687221,-0.103207,-0.999198,-0.335400,0.136468,0.550440,-0.019943,-0.173729
2,2,MOEMPBAHOJKXBG-MRXNPFEDNA-N,-0.751528,-0.506115,0.412968,0.341948,0.822811,-0.713795,0.159594,-0.453231,...,0.530237,-0.131153,-0.007292,-0.065849,-0.978371,-0.653190,0.404358,-0.079914,0.711537,0.445195
3,3,HEKGBDCRHYILPL-QWOVJGMINA-N,-0.757406,0.000328,0.670389,0.856043,0.002886,0.064478,0.181017,-0.229966,...,0.670772,-0.372262,-0.571060,-0.443543,-0.986363,0.118407,-0.077974,-0.097596,0.283461,-0.099510
4,4,SNNRWIBSGBMYRF-UKRRQHHQNA-N,-0.477663,-0.129261,-0.332024,-0.591108,0.786510,-0.007520,-0.171381,-0.048844,...,0.342538,0.680307,0.662410,-0.493203,-0.987440,-0.731436,0.016999,-0.503085,-0.066302,-0.377198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,194,PIKWEFAACQLYMF-UHFFFAOYNA-N,-0.354424,0.037133,0.261493,0.191034,0.203483,-0.718652,0.481088,-0.077800,...,0.465482,0.181667,0.008707,0.374962,-0.998080,-0.015004,-0.071801,-0.205790,-0.394928,0.386006
190,195,AUZWJAMWJZUPHQ-UHFFFAOYNA-N,-0.647606,-0.604185,-0.070398,0.109305,0.667468,-0.239701,-0.332139,-0.490862,...,0.268934,0.103272,-0.120228,-0.133202,-0.981479,-0.683975,0.748666,-0.171097,0.053143,0.144776
191,196,JCEWQICHOLLRDL-WUFINQPMNA-N,-0.681951,-0.346629,0.387501,-0.760321,0.003585,0.173832,0.584196,0.314204,...,-0.401828,0.187382,0.632996,0.507790,-0.999535,0.041612,0.090283,-0.432323,-0.191279,0.136006
192,197,NGRIUVQYFBDXMT-JYAVWHMHNA-N,-0.622850,-0.760069,-0.175192,0.306767,0.828635,-0.251226,0.095201,0.029581,...,-0.428120,0.510929,-0.112762,0.072157,-0.974629,-0.724549,0.754821,0.580699,0.437276,0.079424


The CDDD features are stored in columns `cddd_1`, `cddd_2`, ..., `cddd_512`. The file has the identifier column `Ambit_InchiKey` that we can use to combine the CDDD features with the rest of the data:

In [17]:
def combine_datasets(data, cddd):
    data_combined = pd.merge(
        left=data,
        right=cddd,
        on="Ambit_InchiKey",
        how="inner",
        validate="one_to_one",
    )
    return data_combined

data_combined_train = combine_datasets(data_train, df_cddd)
data_combined_test = combine_datasets(data_test, df_cddd)

Now we can define a pipeline that uses the original SMILES column to compute the descriptors available in scikit-mol, then concatenates them with the pre-computed CDDD features, and uses all of them to train the regression model. We will need a slightly more complex pipeline with column selectors and transformers. For more details on this technique, please refer to the [official documentation](https://scikit-learn.org/stable/modules/generated/sklearn.compose.make_column_selector.html).

Since we will keep everything in DataFrames, it will be easier to understand the effect of the CDDD features and the traditional descriptors available in scikit-mol.

In [18]:
# A pipeline to compute scikit-mol descriptors
descriptors_pipeline = make_pipeline(
    SmilesToMolTransformer(),
    Standardizer(),
    MolecularDescriptorTransformer(),
)
# A pipeline that just passes the input data.
# We will use it to preserve the CDDD features and pass them to downstream steps.
identity_pipeline = make_pipeline(
    FunctionTransformer(),
)
combined_transformer = make_column_transformer(
    (descriptors_pipeline, make_column_selector(pattern="SMILES")),
    (identity_pipeline, make_column_selector(pattern=r"^cddd_\d+$")),
    remainder="drop",
)
combined_transformer

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('smilestomoltransformer',
                                                  SmilesToMolTransformer()),
                                                 ('standardizer',
                                                  Standardizer()),
                                                 ('moleculardescriptortransformer',
                                                  MolecularDescriptorTransformer(desc_list=['MaxAbsEStateIndex',
                                                                                            'MaxEStateIndex',
                                                                                            'MinAbsEStateIndex',
                                                                                            'MinEStateIndex',
                                                                                            'qed',
                                                                                            'SPS',
                                                                                            'MolWt',
                                                                                            'HeavyAtomMolWt',
                                                                                            'ExactMolWt',
                                                                                            'Num...
                                                                                            'BCUT2D_LOGPHI',
                                                                                            'BCUT2D_LOGPLOW',
                                                                                            'BCUT2D_MRHI',
                                                                                            'BCUT2D_MRLOW',
                                                                                            'AvgIpc',
                                                                                            'BalabanJ',
                                                                                            'BertzCT',
                                                                                            'Chi0', ...]))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb6550bf380>),
                                ('pipeline-2',
                                 Pipeline(steps=[('functiontransformer',
                                                  FunctionTransformer())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb6550be090>)])

In [19]:
pipeline_combined = make_pipeline(
    combined_transformer,
    StandardScaler(),
    RandomForestRegressor(**params_random_forest),
)
pipeline_combined.set_output(transform="pandas")

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('smilestomoltransformer',
                                                                   SmilesToMolTransformer()),
                                                                  ('standardizer',
                                                                   Standardizer()),
                                                                  ('moleculardescriptortransformer',
                                                                   MolecularDescriptorTransformer(desc_list=['MaxAbsEStateIndex',
                                                                                                             'MaxEStateIndex',
                                                                                                             'MinAbsEStateIndex',
                                                                                                             'MinEStateIndex',
                                                                                                             'qed',
                                                                                                             'SPS',
                                                                                                             'MolW...
                                                                                                             'Chi0', ...]))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fb6550bf380>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('functiontransformer',
                                                                   FunctionTransformer())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fb6550be090>)])),
                ('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor(max_depth=5))])

In [20]:
pipeline_combined.fit(data_combined_train, target_train)
pred_combined_test = pipeline_combined.predict(data_combined_test)
performance_combined = compute_metrics(y_true=target_test, y_pred=pred_combined_test)
performance_combined

ValueError: Found input variables with inconsistent numbers of samples: [155, 159]

Let's combine the performance metrics obtained using only the scikit-mol descriptors as input features, and the performance metrics obtained using also the CDDD features:

In [ ]:
df_performance = pd.DataFrame([performance, performance_combined], index=["descriptors", "combined"])
df_performance

,RMSE,MAE,R2
descriptors,0.874353,0.726629,0.120818
combined,0.808483,0.673701,0.248295


All performance metrics were improved by the includion of the CDDD features.
Let's analyze the feature importances of the model:

In [ ]:
regressor = pipeline_combined[-1]
df_importance = pd.DataFrame({"feature": regressor.feature_names_in_, "importance": regressor.feature_importances_}).sort_values(by="importance", ascending=False, ignore_index=True)
df_importance

,feature,importance
0,pipeline-1__PEOE_VSA6,0.086083
1,pipeline-2__cddd_102,0.057354
2,pipeline-1__VSA_EState5,0.039388
3,pipeline-2__cddd_378,0.033585
4,pipeline-2__cddd_415,0.032666
...,...,...
717,pipeline-1__fr_hdrzine,0.000000
718,pipeline-1__fr_hdrzone,0.000000
719,pipeline-1__SMR_VSA1,0.000000
720,pipeline-1__fr_imidazole,0.000000


In [ ]:
top_features = df_importance.head(n_top_features).loc[:, "feature"].tolist()
print(f"The {n_top_features} most important features are:")
for feature in top_features:
    print(feature)

The 5 most important features are:
pipeline-1__PEOE_VSA6
pipeline-2__cddd_102
pipeline-1__VSA_EState5
pipeline-2__cddd_378
pipeline-2__cddd_415


As we can see, some CDDD features are among the most important features for the regression model.

Note that since the pipeline is a combination of two pipelines, the column names were prefixed by `pipeline-1` (the scikit-mol descriptors pipeline) and `pipeline-2` (the pipeline that selects and preserves pre-computed CDDD features).